In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.estointernet.in/apache/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf /content/spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


In [ ]:
# urls = 
# {
#     'Music' : 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz',
#     'Instant Video' : 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz',
#     'Office Products' : 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz',
#     'Patio lawn and garden' : 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Patio_Lawn_and_Garden_5.json.gz'
# }

In [ ]:
!curl -L http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz -o data.json.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9294k  100 9294k    0     0  4025k      0  0:00:02  0:00:02 --:--:-- 4027k


In [ ]:
# %fs ls "file:/databricks/driver"

In [ ]:
df = spark.read.json('data.json.gz')
df.show()

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|B000H00VBQ|  [0, 0]|    2.0|I had big expecta...| 05 3, 2014|A11N155CW1UV02|            AdrianaM|A little bit bori...|    1399075200|
|B000H00VBQ|  [0, 0]|    5.0|I highly recommen...| 09 3, 2012|A3BC8O2KCL29V2|             Carol T|Excellent Grown U...|    1346630400|
|B000H00VBQ|  [0, 1]|    1.0|This one is a rea...|10 16, 2013| A60D5HQFOTSOM|Daniel Cooper "da...|Way too boring fo...|    1381881600|
|B000H00VBQ|  [0, 0]|    4.0|Mysteries are int...|10 30, 2013|A1RJPIGRSNX4PW|      J. Kaplan "JJ"|Robson Green is m...|    1383091200|
|B000H00VBQ|  [1, 1]|    5.0|This show always ...|02 11

In [ ]:
# only select the necessary rows
df = df.select('asin','overall','reviewText','reviewerID')
# Convert unique strings to integer using StringIndexer
asin_indexer = StringIndexer(inputCol="asin", outputCol="itemId")
reviewerID_indexer = StringIndexer(inputCol="reviewerID", outputCol="userId")

df = asin_indexer.fit(df).transform(df)
df = reviewerID_indexer.fit(df).transform(df)

In [ ]:
df = df.select('userId','itemId','reviewText','overall')
df = df.withColumnRenamed('overall', 'rating')

df.show()

+------+------+--------------------+------+
|userId|itemId|          reviewText|rating|
+------+------+--------------------+------+
|1962.0|1256.0|I had big expecta...|   2.0|
|4289.0|1256.0|I highly recommen...|   5.0|
| 486.0|1256.0|This one is a rea...|   1.0|
|3445.0|1256.0|Mysteries are int...|   4.0|
|   4.0|1256.0|This show always ...|   5.0|
|  30.0|1256.0|I discovered this...|   5.0|
|2153.0|1433.0|It beats watching...|   3.0|
|2961.0|1433.0|There are many ep...|   3.0|
|4539.0|1433.0|This is the best ...|   5.0|
|1924.0|1433.0|Not bad.  Didn't ...|   3.0|
|1935.0|1433.0|Funny, interestin...|   4.0|
|4258.0|1434.0|I love the variet...|   4.0|
|2462.0|1434.0|comedy is a matte...|   3.0|
|3924.0|1434.0|if this had to do...|   3.0|
|1680.0|1434.0|Watched it for Ke...|   5.0|
|2915.0|1434.0|he's OK. His humo...|   2.0|
|1971.0| 810.0|some comedians ar...|   3.0|
| 599.0| 810.0|I only watched th...|   3.0|
|3924.0| 810.0|Enjoyed some of t...|   5.0|
|4539.0| 810.0|All the comedians

In [ ]:
df = df.withColumn('userId', col('userId').cast('integer')).\
        withColumn('itemId', col('itemId').cast('integer')).\
        withColumn('rating', col('rating').cast('float')).\
        drop('reviewText')

df.show()

+------+------+------+
|userId|itemId|rating|
+------+------+------+
|  1962|  1256|   2.0|
|  4289|  1256|   5.0|
|   486|  1256|   1.0|
|  3445|  1256|   4.0|
|     4|  1256|   5.0|
|    30|  1256|   5.0|
|  2153|  1433|   3.0|
|  2961|  1433|   3.0|
|  4539|  1433|   5.0|
|  1924|  1433|   3.0|
|  1935|  1433|   4.0|
|  4258|  1434|   4.0|
|  2462|  1434|   3.0|
|  3924|  1434|   3.0|
|  1680|  1434|   5.0|
|  2915|  1434|   2.0|
|  1971|   810|   3.0|
|   599|   810|   3.0|
|  3924|   810|   5.0|
|  4539|   810|   5.0|
+------+------+------+
only showing top 20 rows



In [ ]:
paramRank = [5, 10, 25, 50]
paramReg = [0.1, 0.01, 0.001]

maxIter = 20
iter_list = [i for i in range(1,maxIter+1)]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import time

(train, test) = df.randomSplit([0.8, 0.2], seed = 1234)


for rank in paramRank:
  for reg in paramReg:
    
    print(f"RMSE for rank: {rank} reg: {reg} maxiter: {maxIter}")
    # measuring time of training
    start_time = time.time()
    als = ALS(maxIter=maxIter, regParam=reg, rank=rank,userCol="userId", itemCol="itemId", ratingCol="rating",coldStartStrategy="drop")
    model = als.fit(train) 
    # Evaluate the model by computing the RMSE on the test data
    predictions = model.transform(test)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
    rmse = evaluator.evaluate(predictions)
    total_time = time.time() - start_time
    
    print(f"RMSE = {rmse} time = {total_time}")      

RMSE for rank: 5 reg: 0.1 maxiter: 20
RMSE = 1.2175521436359544 time = 20.694786548614502
RMSE for rank: 5 reg: 0.01 maxiter: 20
RMSE = 4.74055840411915 time = 16.83229160308838
RMSE for rank: 5 reg: 0.001 maxiter: 20
RMSE = 12.806468950136264 time = 17.17692470550537
RMSE for rank: 10 reg: 0.1 maxiter: 20
RMSE = 1.1716486303246794 time = 16.293590545654297
RMSE for rank: 10 reg: 0.01 maxiter: 20
RMSE = 3.091716529691959 time = 16.230996131896973
RMSE for rank: 10 reg: 0.001 maxiter: 20
RMSE = 6.248090327350265 time = 16.491925954818726
RMSE for rank: 25 reg: 0.1 maxiter: 20
RMSE = 1.1172076641934812 time = 18.045958042144775
RMSE for rank: 25 reg: 0.01 maxiter: 20
RMSE = 1.3656537679984035 time = 18.468995332717896
RMSE for rank: 25 reg: 0.001 maxiter: 20
RMSE = 2.2796640042535343 time = 17.89062762260437
RMSE for rank: 50 reg: 0.1 maxiter: 20
RMSE = 1.1049291362255416 time = 24.725177764892578
RMSE for rank: 50 reg: 0.01 maxiter: 20
RMSE = 1.2672485685849209 time = 24.884576320648193